In [2]:
import spacy 
#from spacy.lang.en import English
#from spacy.pipeline import EntityRuler
import json
import random

In [ ]:
#don't mess with the codeblock box
def load_data(file):
    with open(file, 'r', encoding='utf-8') as f:
        #print(f)
        return(json.load(f))
    
    
def save_data (file, data):
    with open (file, "w", encoding='utf-8') as f:
       data = json.dump(data, f, indent=4)
       #print(data)

def create_training_data(file, type):
    #print(type)
    #print(file)
    data = load_data(file)
    #print(data)
    patterns=[]
    for key, value in data.items():
        #print(key, value)
        for obj in value: #muni loop
            #print(obj.get("Municipality"))
            pattern = {
                "label":type,
                "pattern": obj.get("Municipality")
                }
            patterns.append(pattern) # end of muni loop
        for obj in value: #primary county loop 
        #if any(d.get("Municipality")=='Karnes' for d in value): 
            #print("found") 
            if not any(d.get("Municipality")== obj.get("Primary County") for d in value):
                print('not found')
                pattern = {
                    "label":type,
                    "pattern": obj.get("Primary County")
                    }
                patterns.append(pattern)
                #if any(d.get("Primary County")=='Karnes' for d in value): 
                    #print("found in pc")
        
            #print(pattern)
            #print(patterns)
    return(patterns)
patterns = create_training_data('../Corridos/data/TexasMunicipalities.json', 'GPE')
print(patterns)

In [ ]:
#don't mess with this codeblock box
#TX NER TRAINING SET 
def load_data(file):
    with open(file, 'r', encoding='utf-8') as f:
        #print(f)
        return(json.load(f))
     
def save_data (file, data):
    with open (file, "w", encoding='utf-8') as f:
       json.dump(data, f, indent=4)
       #print(data)

def create_training_data(file, type):
    #print(type)
    #print(file)
    data = load_data(file)
    #print(data)
    patterns=[]
    for key, value in data.items():
        #print(key, value)
        for obj in value: #muni loop
            #print(obj.get("Municipality"))
            pattern = {
                "label":type,
                "pattern": obj.get("Municipality")
                }
            patterns.append(pattern) # end of muni loop
        for obj in value: #primary county loop 
        #if any(d.get("Municipality")=='Karnes' for d in value): 
            #print("found") 
            if not any(d.get("Municipality")== obj.get("Primary County") for d in value):
                #print('not found')
                pattern = {
                    "label":type,
                    "pattern": obj.get("Primary County")
                    }
                patterns.append(pattern)
                #if any(d.get("Primary County")=='Karnes' for d in value): 
                    #print("found in pc")
        
            #print(pattern)
            #print(patterns)
        return(patterns)


#patterns = create_training_data('../Corridos/data/TexasMunicipalities.json', 'GPE')
#generate_rules(patterns)

def test_model(model, text):
    doc = nlp(text)
    results = []
    entities = []
    for ent in doc.ents:
        entities.append((ent.start_char, ent.end_char, ent.label_))
    if len(entities) >0: #if entity has been found move entity to results and entity list
        results = [text, {"entities": entities}]
        #print(results)
    return (results)

#how spacy wants to see read the data#
#TRAIN_DATA = [(text, {"entities": [(start, end, label)]})]#


nlp = spacy.load("tx_trained_ner")
TRAIN_DATA= []
with open ("../Corridos/corrido corpus/ElCorridodeGregorioCortez_X.txt", "r", encoding='utf-8')as f:
    text = f.read()
    #print(text)
    segments = text.split("\n\n")[0:]
    #cleaning up the lyric text. making it easier to read for the program
    for segment in segments:
        #segment = segment.strip() #might not need this code line 
        #segment = segement.replace("\n", " ")# might not need this code line
        #print(segment)
        results = test_model(nlp, segment)
        if results != None: #it found something and return it
            TRAIN_DATA.append(results)
        #print(results)

print(len(TRAIN_DATA))
#save_data("../Corridos/data/TxMunicipality_training_data.json", TRAIN_DATA)

In [32]:
def load_data(file):
    with open(file, 'r', encoding='utf-8') as f:
        #print(f)
        return(json.load(f))
    
    
def save_data (file, data):
    with open (file, "w", encoding='utf-8') as f:
       json.dump(data, f, indent=4)
       #print(data)

def train_spacy(data, iterations): #passing training data/set and the number/generation of the training process. 30 is good. long time!!!!
    #print(data)
    TRAIN_DATA = data
    #print(TRAIN_DATA)
    nlp = spacy.blank("en")
    #print(nlp)
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner") #if there is no ner in the pipe, create a pipe
        nlp.add_pipe("ner", last=True)
    
    for _, annotations in TRAIN_DATA: #add labels
        #print(annotations)
        for ent in annotations.get("entities"):
            ner.add_label(ent[2]) # i'm only working with one label 'GPE' if more increase i believe double check
    
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"] #get other names of other pipes to disable them during training. 
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()
        for itn in range(iterations): # adjust the funtion not the items inside. arg of interations 30 is good
            print("Starting iteration "+ str(itn)) #where I am at in the interations process
            random.shuffle(TRAIN_DATA) #shuffle helps the program to learn not memorize order. 
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                            (text,
                            annotations),
                            drop=0.2, #to help avoid over fitting which is a common practice in machine learning. 
                            sgd= optimizer,
                            losses=losses
                            )
            print(losses)  
    return(nlp) #model return

TRAIN_DATA = load_data("../Corridos/data/TxMunicipality_training_data.json")
print(TRAIN_DATA)
nlp = train_spacy(TRAIN_DATA, 30)
nlp.to_disk("tx_ner_model")


[['In the country of Karnes, \nLook what has happened; \nThe Major Sheriff died,\nLeaving Román badly wounded.', {'entities': [[18, 24, 'GPE']]}], [], [], [], [], [], [], [], [], ['He struck out for Gonzales\nWithout showing any fear,\n“Follow me, cowardly rangers, \nI am Gregorio Cortez.”', {'entities': [[18, 26, 'GPE']]}], [], [], [], ['Gregorio Cortez went out, \nHe went towards Laredo\nThey decided not to follow \nBecause they were afraid of him.', {'entities': [[43, 49, 'GPE']]}], [], [], ['Over by El Encinal, \nAccording to what we hear, \nThey made him a corral, \nAnd he killed them another sheriff. ', {'entities': [[11, 18, 'GPE']]}], [], [], [], [], [], [], [], [], [], [], []]


ValueError: not enough values to unpack (expected 2, got 0)